In [1]:
import numpy as np
import pandas as pd
import re

In [11]:
# how to read a pickle file

# tracks_features_processed is a dataframe with danceability and all the features
# we want. It is tied to an id, which is not the same as the one
# as the track id from the playlist dataset we will access below using
# json
tracks_features_processed = pd.read_pickle('tracks_features_processed.pkl')

# track_identifier holds the id and the shared features with the playlist
# dataset below. I think we should use this to join the two tables together
track_identifier = pd.read_pickle('track_identifier.pkl')

In [12]:
# tracks_features_processed.head(10)
track_identifier.head(10)

,id,name,album,artists,duration_ms
0,7lmeHLHBe4nmXzuXc0HDjk,testify,thebattleoflosangeles,[Rage Against The Machine],210133
1,1wsRitfRRtWyEapl0q22o8,guerrillaradio,thebattleoflosangeles,[Rage Against The Machine],206200
2,1hR0fIFK2qRG3f3RF70pb7,calmlikeabomb,thebattleoflosangeles,[Rage Against The Machine],298893
3,2lbASgTSoDO7MTuLAXlTW0,miccheck,thebattleoflosangeles,[Rage Against The Machine],213640
4,1MQTmpYOZ6fcMQc56Hdo7T,sleepnowinthefire,thebattleoflosangeles,[Rage Against The Machine],205600
5,2LXPNLSMAauNJfnC58lSqY,bornofabrokenman,thebattleoflosangeles,[Rage Against The Machine],280960
6,3moeHk8eIajvUEzVocXukf,bornasghosts,thebattleoflosangeles,[Rage Against The Machine],202040
7,4llunZfVXv3NvUzXVB3VVL,maria,thebattleoflosangeles,[Rage Against The Machine],228093
8,21Mq0NzFoVRvOmLTOnJjng,voiceofthevoiceless,thebattleoflosangeles,[Rage Against The Machine],151573
9,6s2FgJbnnMwFTpWJZzvb6z,newmillenniumhomes,thebattleoflosangeles,[Rage Against The Machine],224933


In [13]:
# TRACK_IDENTIFIER PREPROCESSING
track_identifier['name'] = track_identifier['name'].apply(lambda row: re.sub(r'[^A-Za-z0-9 ]+', '', row))
track_identifier['album'] = track_identifier['album'].apply(lambda row: re.sub(r'[^A-Za-z0-9 ]+', '', row))
track_identifier['name'] = track_identifier['name'].str.lower()
track_identifier['album'] = track_identifier['album'].str.lower()
track_identifier['name'] = track_identifier['name'].apply(lambda row: re.sub(r'\s+', '', row))
track_identifier['album'] = track_identifier['album'].apply(lambda row: re.sub(r'\s+', '', row))
track_identifier.head(15)
track_identifier.to_pickle('track_identifier.pkl')

In [38]:
# playlist = pd.read_json('spotify_million_playlist_dataset/data/mpd.slice.0-999.json')
import json
# load data using Python JSON module
with open('spotify_million_playlist_dataset/data/mpd.slice.0-999.json','r') as f:
    data = json.loads(f.read())
    
# Normalizing data
df = pd.json_normalize(data, record_path =['playlists'])
print(df.columns)
df = df[['pid','name','tracks','num_tracks']]
df.head()

Index(['name', 'collaborative', 'pid', 'modified_at', 'num_tracks',
       'num_albums', 'num_followers', 'tracks', 'num_edits', 'duration_ms',
       'num_artists', 'description'],
      dtype='object')


,pid,name,tracks,num_tracks
0,0,Throwbacks,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",52
1,1,Awesome Playlist,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",39
2,2,korean,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",64
3,3,mat,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",126
4,4,90s,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",17


In [39]:
def convert_to_tracks(tracks_dict):
    tracks = []
    for track in tracks_dict:
        curr_artist = track['artist_name']
        curr_artist = re.sub(r'[^A-Za-z0-9 ]+', '', curr_artist)
        curr_artist = re.sub(r'\s+', '', curr_artist)
        curr_album = track['album_name']
        curr_album = re.sub(r'[^A-Za-z0-9 ]+', '', curr_album)
        curr_album = re.sub(r'\s+', '', curr_album)
        curr_duration = track['duration_ms']
        curr_artist = curr_artist.lower()
        curr_album = curr_album.lower()
        # curr_track = track['track_name']
        # curr_track = re.sub(r'\[[^)]*\]', '', curr_track)
        # tmp_df = track_identifier[track_identifier['name'].str.contains(curr_track)].copy()
        tmp_df = track_identifier[track_identifier['album'].str.contains(curr_album)].copy()
        tmp_df = tmp_df[tmp_df['duration_ms'] < curr_duration + 100].copy()
        tmp_df = tmp_df[tmp_df['duration_ms'] > curr_duration - 100].copy()
        # tmp_df = track_identifier[track_identifier['name'].str.contains(curr_track)].copy()
        # print(list(tmp_df['id']))
        if len(list(tmp_df['id'])) > 0:
            tracks.append(list(tmp_df['id'])[0])
    return tracks

def debug_convert_to_tracks(tracks_dict):
    tracks = []
    counter = 0
    for track in tracks_dict:
        print(counter)
        # print(track)
        curr_artist = track['artist_name']
        curr_album = track['album_name']
        curr_duration = track['duration_ms']
        curr_track = track['track_name']
        # curr_track = re.sub(r'\[[^)]*\]', '', curr_track)
        # print(curr_track)
        print(curr_album)
        print(curr_artist)
        # tmp_df = track_identifier[track_identifier['name'].str.contains(curr_track)].copy()
        tmp_df = track_identifier[track_identifier['album'].str.contains(curr_album)].copy()
        # print(tmp_df)
        tmp_df = tmp_df[tmp_df['duration_ms'] < curr_duration + 500].copy()
        tmp_df = tmp_df[tmp_df['duration_ms'] > curr_duration - 500].copy()
        tmp_df = tmp_df[tmp_df['name'].str.contains(curr_track)].copy()
        print(tmp_df)
        # print(list(tmp_df['id']))
        if len(list(tmp_df['id'])) > 0:
            print('\ntrack to be added: ', list(tmp_df['id'])[0])
            tracks.append(list(tmp_df['id'])[0])
        counter += 1
    return tracks

In [40]:
for i in range(len(df)):
    print(i)
    df['tracks'].iloc[i] = convert_to_tracks(df['tracks'].iloc[i])

0
1


/var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/ipykernel_7302/38845374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tracks'].iloc[i] = convert_to_tracks(df['tracks'].iloc[i])


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202


KeyboardInterrupt: 

In [41]:
df['num_tracks_id'] = df['tracks'].apply(lambda row: len(row))

In [43]:
df.head(40)
# track_identifier.shape

,pid,name,tracks,num_tracks,num_tracks_id
0,0,Throwbacks,"[6I9VzXrHxO9rA9A5euc8Ak, 1AWQoqb9bSvzTjaLralEk...",52,12
1,1,Awesome Playlist,"[2HHtWyy5CgaQbC7XSoOb0e, 2TK2KSrzXD6W01qjXVjNG...",39,12
2,2,korean,"[0WgvYSbwM0c8KUq8KTrraB, 0BDU0LvLow4jycHSNpRA8...",64,10
3,3,mat,"[4WJ7UMD4i6DOPzyXU5pZSz, 2K0HAp3di1AvmIjYYdPk4...",126,21
4,4,90s,"[6qspW4YKycviDFjHBOaqUY, 1Jaah2tmN9Hv81A87KZ1M...",17,7
5,5,Wedding,"[0CAfXk7DXMnon4gLudAp7J, 5uuJruktM9fMdN9Va0DUM...",80,11
6,6,I Put A Spell On You,"[4N3MCeo4dfVw8vFCjV9uJl, 2s1sdSqGcKxpPr5lCl7jA...",16,4
7,7,2017,"[23IF3ytLVAz2ooolWtUjQQ, 7mQIfLReGsYu369bKZx9b...",53,7
8,8,BOP,"[2XTkpF9T2PKvcLgamGJGx1, 0MhqQfM967D8Mu5HHT8M2...",46,12
9,9,old country,"[6gXuwld0PRtinePSXV2Vn1, 3oW6SWwGqiZSPTiAp7ZQo...",21,7


In [44]:
df.to_pickle('temporary_tracks.pkl')